<a href="https://colab.research.google.com/github/islamicity24/PythonCity/blob/main/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [2]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [3]:
# Display query output immediately

%%bigquery --project dashboard-project-388011
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

Query is running:   0%|          |

Downloading:   0%|          |

,total_rows
0,114420316


In [4]:
# Save output in a variable `df`

%%bigquery df --project dashboard-project-388011
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=gsod&page=table) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [6]:
project_id = 'dashboard-project-388011'

### Sample approximately 2000 random rows

In [7]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [8]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,min_temperature,total_precipitation,snow_depth
count,2004.0,2004.0,2004.0,2004.0,2004.0,2004.000000,2004.0,1911.000000,1911.0,1512.000000,...,1793.000000,1793.0,1982.000000,1982.0,1944.000000,249.000000,2001.000000,0.0,1818.000000,108.000000
mean,508773.778942,90940.555389,1987.052395,6.561377,15.425649,51.729092,12.731038,41.235322,12.680272,1014.860120,...,12.416787,12.327384,6.692281,12.684157,12.093467,24.845381,43.107696,NaN,0.063531,10.871296
std,299169.780512,25703.376495,16.151762,3.488334,8.890242,24.594463,7.888389,23.178137,7.895709,9.712478,...,9.442823,7.763052,5.027897,7.877478,6.845413,9.082111,24.728250,NaN,0.253602,12.326971
min,10010.0,6.0,1937.0,1.0,1.0,-58.700001,4.0,-57.799999,4.0,901.200012,...,0.000000,4.0,0.000000,4.0,1.000000,9.700000,-68.400002,NaN,0.000000,0.400000
25%,257710.0,99999.0,1977.0,3.0,8.0,37.599998,7.0,28.900000,6.0,1009.500000,...,6.300000,6.0,3.300000,6.0,7.800000,18.799999,30.900000,NaN,0.000000,2.000000
50%,528965.0,99999.0,1989.0,7.0,15.0,55.200001,8.0,43.500000,8.0,1014.400024,...,9.900000,8.0,5.600000,8.0,11.100000,23.299999,46.000000,NaN,0.000000,7.900000
75%,725151.25,99999.0,2000.0,10.0,23.0,70.425001,23.0,57.500000,23.0,1020.200012,...,14.900000,22.0,8.775000,23.0,15.000000,29.100000,60.799999,NaN,0.010000,15.000000
max,999999.0,99999.0,2010.0,12.0,31.0,99.000000,24.0,81.500000,24.0,1054.199951,...,99.400002,24.0,41.299999,24.0,62.200001,66.000000,90.099998,NaN,6.100000,80.699997


### View the first 10 rows

In [9]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,749211,99999,1942,11,28,0.800000,24,-5.400000,24,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
1,999999,14775,1950,1,19,22.100000,24,6.400000,24,1021.000000,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
2,722485,13944,1961,3,11,56.900002,24,46.000000,24,1019.500000,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
3,560340,99999,1964,11,9,17.299999,7,-2.700000,7,NaN,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
4,999999,13883,1964,8,9,79.699997,24,68.199997,24,1015.599976,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
5,314780,99999,1970,10,15,8.800000,6,-2.300000,6,1014.400024,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
6,722420,99999,1975,2,15,63.500000,4,60.299999,4,1011.299988,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
7,725295,99999,1977,5,13,57.599998,12,NaN,<NA>,1009.500000,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
8,471380,99999,1980,12,2,42.000000,7,32.000000,7,1014.299988,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
9,235890,99999,1984,6,19,58.000000,4,47.500000,4,1007.000000,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False


In [10]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
1195,429710,1991,7,28,6.10
1218,76840,1999,10,23,2.83
790,654200,2009,4,2,2.48
1850,566910,1974,10,3,2.24
658,479360,1986,8,29,2.09
375,842030,1982,9,26,2.09
591,476680,1961,3,19,2.01
1453,590210,1997,4,3,1.97
1202,484390,2009,7,21,1.69
1717,589270,1976,2,28,1.57


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [11]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [12]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
